In [153]:
# import the necessary libraries
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.core.display import Markdown
from IPython.display import  display
from anthropic import Anthropic

In [154]:
# load the environment variables
load_dotenv(override=True)

True

In [155]:
# check if the environment variables are set
openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY is not set")
if anthropic_api_key is None:
    raise ValueError("ANTHROPIC_API_KEY is not set")
if google_api_key is None:
    raise ValueError("GOOGLE_API_KEY is not set")

print(openai_api_key[0:5])
print(anthropic_api_key[0:5])
print(google_api_key[0:5])



sk-pr
sk-an
AIzaS


In [156]:
request = "Please come up with a challenging , nuanced question that I can ask a number of LLMs to evaluate their intelligence."
request+= " Answer only with the question , no explanaition."
messages = [{"role": "user", "content": request}]


In [157]:
messages

[{'role': 'user',
  'content': 'Please come up with a challenging , nuanced question that I can ask a number of LLMs to evaluate their intelligence. Answer only with the question , no explanaition.'}]

In [158]:
openai = OpenAI(api_key=openai_api_key)
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
)

question = response.choices[0].message.content
print(question)

What is the most effective strategy for balancing the rights of indigenous communities with the goals of economic development in the context of natural resource extraction?


In [159]:
competitors = []
answers = []
messages = [{"role": "user", "content": question}]

In [160]:
# Gpt Model
model_name = "gpt-4o-mini"

repsonse = openai.chat.completions.create(model=model_name, messages=messages)
answer = repsonse.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Balancing the rights of indigenous communities with the goals of economic development in the context of natural resource extraction requires a multifaceted strategy that prioritizes respect, collaboration, and sustainability. Here are several effective strategies:

1. **Free, Prior, and Informed Consent (FPIC):** Ensure that indigenous communities are fully informed about potential resource extraction projects and have the right to give or withhold consent before these projects proceed. This respect for autonomy helps build trust and ensures that indigenous voices are integral to decision-making.

2. **Stakeholder Involvement:** Create inclusive platforms for dialogue among all stakeholders, including government agencies, businesses, and indigenous groups. Collaborative decision-making processes can lead to better outcomes and greater acceptance of development projects.

3. **Ecosystem-Based Management:** Implement strategies that prioritize the ecological health of the land over short-term economic gains. This can involve sustainable practices that align the interests of economic development with environmental stewardship, which is often a priority for indigenous communities.

4. **Benefit Sharing:** Develop frameworks that ensure indigenous communities receive direct economic benefits from resource extraction projects. This can include profit-sharing agreements, revenue-sharing models, and funding for community infrastructure and services.

5. **Cultural Heritage Protection:** Safeguard the cultural, spiritual, and historical sites that may be affected by resource development. Implement measures to preserve and protect these sites while allowing for economic activities to occur.

6. **Capacity Building:** Support indigenous communities in building their capacity to engage in negotiations and partnerships regarding resource extraction. This can include legal training, business development support, and education initiatives.

7. **Regulatory Frameworks:** Enforce strong legal protections for indigenous rights at national and international levels. Implement regulations that specifically require consideration of indigenous rights in environmental assessments and development permits.

8. **Conflict Resolution Mechanisms:** Establish clear and fair mechanisms for resolving disputes that may arise between indigenous communities and developers. These mechanisms should be accessible and culturally appropriate.

9. **Long-Term Partnerships:** Promote long-term partnerships between indigenous groups and companies focused on sustainable practices. Such partnerships can lead to innovative solutions that respect indigenous rights while meeting development goals.

10. **Monitoring and Accountability:** Ensure ongoing monitoring of resource extraction impacts on indigenous communities and ecosystems, with mechanisms for accountability when commitments are not met or when negative impacts are observed.

By integrating these strategies, it is possible to foster a balanced approach that respects the rights and aspirations of indigenous communities while pursuing economic development through natural resource extraction. This requires ongoing commitment from all parties to engage respectfully and collaboratively.

In [161]:
# Anthropic model
model_name = "claude-3-7-sonnet-20250219"

claude = Anthropic()
response = claude.messages.create(model=model_name, messages=messages, max_tokens=1000)
answer = response.content[0].text

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

# Balancing Indigenous Rights and Economic Development

The most effective strategy requires a multi-faceted approach that recognizes there's no single universal solution:

**Core principles that should guide any approach:**

1. **Free, Prior, and Informed Consent (FPIC)** - Ensuring indigenous communities can give or withhold consent to projects affecting their lands and resources

2. **Benefit-sharing arrangements** - Creating mechanisms where indigenous communities receive equitable portions of economic benefits

3. **Co-management frameworks** - Involving indigenous peoples in decision-making and environmental monitoring

4. **Cultural impact assessments** alongside environmental ones

5. **Legal recognition of indigenous land rights** as a foundation

The most successful approaches tend to be collaborative models that view indigenous communities as partners rather than obstacles. This means early engagement, transparent processes, and recognition that economic development and indigenous rights can be complementary rather than contradictory goals.

Different contexts may require different specific implementations, but these principles provide a framework that respects both development needs and indigenous sovereignty.

In [162]:
# Gemini model
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"
response = gemini.chat.completions.create(model=model_name, messages=messages)

answer = response.choices[0].message.content
display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)



Balancing the rights of indigenous communities with economic development goals in natural resource extraction is a complex and sensitive issue. There's no single "most effective" strategy, as each situation is unique and requires a tailored approach. However, several key principles and strategies, when implemented in combination, can significantly improve outcomes:

**1. Free, Prior, and Informed Consent (FPIC):**

*   **What it is:** This is the cornerstone of ethical and sustainable resource extraction. It means that indigenous communities have the right to:
    *   **Free:** No coercion or manipulation.
    *   **Prior:**  Consent sought *before* any project begins and throughout its lifecycle.
    *   **Informed:** Access to comprehensive information in a culturally appropriate and accessible format about the potential impacts (positive and negative) of the project.
    *   **Consent:** The right to say 'yes' or 'no' to the project.  While not a veto in all jurisdictions, lack of consent should trigger serious re-evaluation.
*   **Why it's important:** Respects self-determination, empowers communities to participate in decision-making, and leads to more equitable outcomes.
*   **Challenges:** Can be difficult to implement effectively, particularly with diverse communities or in situations where power imbalances exist.  Needs robust legal frameworks and oversight.

**2. Strengthening Indigenous Governance and Capacity Building:**

*   **What it is:** Investing in the capacity of indigenous communities to effectively engage in negotiations, manage resources, and participate in economic development.  This includes:
    *   Supporting traditional leadership structures and decision-making processes.
    *   Providing education and training in areas like negotiation, business management, environmental monitoring, and legal rights.
    *   Strengthening land tenure security and legal frameworks recognizing indigenous rights.
*   **Why it's important:**  Ensures communities have the knowledge and skills to advocate for their interests, manage their resources sustainably, and benefit from economic development.
*   **Challenges:** Requires long-term commitment and culturally sensitive approaches. Funding and resources must be directed to community-led initiatives.

**3. Benefit-Sharing Agreements:**

*   **What it is:**  Formal agreements that outline how indigenous communities will benefit from resource extraction projects.  This can include:
    *   Royalties and revenue sharing.
    *   Employment and training opportunities.
    *   Procurement opportunities for local businesses.
    *   Community development projects (e.g., schools, health clinics, infrastructure).
    *   Environmental protection measures.
*   **Why it's important:** Provides direct economic benefits to communities, improves living standards, and fosters a sense of ownership and partnership.
*   **Challenges:**  Ensuring agreements are fair, transparent, and enforceable.  Requires careful negotiation and ongoing monitoring.

**4. Environmental and Social Impact Assessments (ESIAs) with Meaningful Participation:**

*   **What it is:** Thorough assessments conducted *before* projects begin to identify potential environmental and social impacts, including impacts on indigenous cultures, livelihoods, and sacred sites.  Crucially, indigenous communities must be actively involved in all stages of the EIA process, from scoping to monitoring.
*   **Why it's important:**  Helps to mitigate negative impacts, avoid irreversible damage, and ensure that projects are environmentally and socially responsible.
*   **Challenges:**  ESIA's can be expensive and time-consuming.  They also need to be independent and free from bias.  Indigenous knowledge must be integrated into the assessment process.

**5. Independent Monitoring and Grievance Mechanisms:**

*   **What it is:**  Establishing independent bodies to monitor the implementation of agreements, environmental performance, and social impacts of resource extraction projects.  Developing accessible and effective grievance mechanisms for communities to raise concerns and seek redress.
*   **Why it's important:**  Ensures accountability, transparency, and responsiveness to community concerns.
*   **Challenges:**  Ensuring independence and impartiality of monitoring bodies.  Developing culturally appropriate and accessible grievance mechanisms.

**6. Policy and Legal Frameworks that Recognize and Protect Indigenous Rights:**

*   **What it is:**  Developing strong legal frameworks that recognize indigenous land rights, self-determination, and the right to FPIC.  Implementing policies that promote sustainable development and environmental protection.
*   **Why it's important:**  Provides a legal basis for protecting indigenous rights and ensuring that resource extraction is conducted in a responsible manner.
*   **Challenges:**  Overcoming historical injustices and discriminatory laws.  Ensuring that legal frameworks are enforced effectively.

**7. Cultural Sensitivity and Respect:**

*   **What it is:**  Adopting a culturally sensitive approach that recognizes and respects indigenous cultures, traditions, and values.  Engaging with communities in a respectful and equitable manner.
*   **Why it's important:**  Builds trust, fosters collaboration, and avoids cultural insensitivity.
*   **Challenges:**  Requires ongoing learning and adaptation.  Companies and governments need to invest in cultural awareness training.

**In summary, the most effective strategy is a holistic approach that incorporates:**

*   **Respect for Indigenous Rights:** Placing indigenous rights at the forefront of decision-making.
*   **Genuine Partnership:** Building meaningful partnerships with indigenous communities based on trust and mutual respect.
*   **Transparency and Accountability:** Ensuring transparency in all stages of the project and holding companies and governments accountable for their actions.
*   **Long-Term Sustainability:**  Focusing on sustainable development that benefits both indigenous communities and the wider economy over the long term.

**It's also important to remember that:**

*   **Context Matters:**  Each situation is unique, and the best approach will vary depending on the specific context, including the cultural, social, economic, and environmental characteristics of the area.
*   **Continuous Improvement:**  The process of balancing indigenous rights and economic development is ongoing and requires continuous learning, adaptation, and improvement.
*   **Political Will:** The successful implementation of these strategies requires strong political will from governments and a genuine commitment from companies to respect indigenous rights and promote sustainable development.

By adopting these principles and strategies, we can work towards a more just and equitable future where indigenous communities can thrive alongside economic development.


## How to use Ollama

Ollama is a tool that allows you to run large language models locally on your machine, providing an OpenAI-compatible API endpoint for easy integration.

### Step 1: Installation
1. Visit [https://ollama.com/download](https://ollama.com/download)
2. Download the Windows version
3. Run the installer and follow the instructions
4. Restart your terminal/command prompt after installation

### Step 2: Start the server
After installation, Ollama usually starts automatically as a background service. To verify it's running:
- Visit [http://localhost:11434](http://localhost:11434) 
- You should see the message: "Ollama is running"

### Step 3: Download and use models
```bash
# Download a model (this may take several minutes)
ollama pull llama3.2

# Run a model interactively
ollama run llama3.2

# Test with a quick prompt
ollama run llama3.2 "Hello, how are you?"
```

## Essential Ollama Commands

### Model Management
- `ollama pull <model_name>` - Download a specific model from the library
- `ollama ls` or `ollama list` - List all installed models with their sizes
- `ollama rm <model_name>` - Remove a specific model to free up disk space
- `ollama show <model_name>` - Display detailed information about a model

### Server Management
- `ollama serve` - Start the Ollama server manually (usually not needed)
- `ollama ps` - Show currently running models and their status
- `ollama stop <model_name>` - Stop a specific running model

### Interactive Usage
- `ollama run <model_name>` - Start an interactive chat session with a model
- `ollama run <model_name> "your prompt"` - Get a single response from a model

### Popular Models to Try
- `llama3.2` - Latest Llama model (3B or 1B parameters)
- `llama3.2:70b` - Larger Llama model for better performance
- `mistral` - Efficient 7B parameter model
- `codellama` - Specialized for code generation
- `gemma2` - Google's open model

### API Usage
Once Ollama is running, you can use it with the OpenAI-compatible API at:
- Base URL: `http://localhost:11434/v1`
- No API key required for local usage

In [163]:
ollama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
model_name = "llama3.2"
response = ollama.chat.completions.create(model=model_name, messages=messages)

answer = response.choices[0].message.content
display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Balancing the rights of indigenous communities with the goals of economic development in the context of natural resource extraction requires a multi-faceted approach that takes into account the needs and concerns of all stakeholders. Here are some effective strategies:

1. **Free, Prior, and Informed Consent (FPIC)**: FPIC is a widely accepted principle that requires companies to obtain consent from indigenous communities before engaging in extractive activities on their land or territory. This involves providing detailed information about the project's impact, potential risks, and benefits.
2. **Participatory Planning**: Engage indigenous communities in the planning process through inclusive consultations, participatory budgeting, and other forms of co-decision-making. This helps ensure that community concerns are taken into account and their needs are addressed.
3. **Capacity Building**: Support indigenous communities to develop their capacity to participate in decision-making processes, negotiate with companies, and access legal and technical resources.
4. **Land Rights Protection**: Recognize and protect indigenous land rights through clear and effective legislation, such as the United Nations Declaration on the Rights of Indigenous Peoples (UNDRIP).
5. **Community Engagement**: Foster strong relationships between extractive companies and local communities by engaging with community representatives in a transparent, inclusive, and respectful manner.
6. **Monitoring and Oversight**: Establish robust monitoring and oversight mechanisms to ensure compliance with FPIC regulations, environmental standards, and human rights norms.
7. **Collaborative Governance**: Encourage collaboration between governments, industry, and civil society organizations to develop effective policies and frameworks that balance economic development with indigenous rights.
8. **Inclusive Benefit-Sharing**: Implement benefit-sharing schemes that prioritize community participation and benefit distribution, ensuring that local populations receive fair share of economic benefits from extractive activities.
9. **Environmental and Social Impact Assessments (ESIAs)**: Conduct thorough ESIAs to assess potential environmental and social impacts on indigenous communities and develop mitigation strategies to minimize harm.
10. **Stakeholder Engagement**: Foster ongoing dialogue among stakeholders, including local communities, industry representatives, governments, civil society organizations, and international financial institutions, to ensure that concerns are addressed and progress is made.

Effective balance between economic development goals and indigenous rights requires:

1. A willingness to listen and adapt
2. Capacity building and empowerment of local communities
3. Partnerships and collaborations among stakeholders
4. Strong governance structures in place to oversee the implementation of decisions
5. Inclusive benefit-sharing mechanisms
6. Environmental and social monitoring systems

In [164]:

print("competitors:", competitors)
print(answers)

competitors: ['gpt-4o-mini', 'claude-3-7-sonnet-20250219', 'gemini-2.0-flash', 'llama3.2']
['Balancing the rights of indigenous communities with the goals of economic development in the context of natural resource extraction requires a multifaceted strategy that prioritizes respect, collaboration, and sustainability. Here are several effective strategies:\n\n1. **Free, Prior, and Informed Consent (FPIC):** Ensure that indigenous communities are fully informed about potential resource extraction projects and have the right to give or withhold consent before these projects proceed. This respect for autonomy helps build trust and ensures that indigenous voices are integral to decision-making.\n\n2. **Stakeholder Involvement:** Create inclusive platforms for dialogue among all stakeholders, including government agencies, businesses, and indigenous groups. Collaborative decision-making processes can lead to better outcomes and greater acceptance of development projects.\n\n3. **Ecosystem-Ba

In [165]:
# Compile all answers together
for competitor, answer in zip(competitors, answers):
    print(f"## {competitor}\n")
    print(answer)
    print("\n\n")

## gpt-4o-mini

Balancing the rights of indigenous communities with the goals of economic development in the context of natural resource extraction requires a multifaceted strategy that prioritizes respect, collaboration, and sustainability. Here are several effective strategies:

1. **Free, Prior, and Informed Consent (FPIC):** Ensure that indigenous communities are fully informed about potential resource extraction projects and have the right to give or withhold consent before these projects proceed. This respect for autonomy helps build trust and ensures that indigenous voices are integral to decision-making.

2. **Stakeholder Involvement:** Create inclusive platforms for dialogue among all stakeholders, including government agencies, businesses, and indigenous groups. Collaborative decision-making processes can lead to better outcomes and greater acceptance of development projects.

3. **Ecosystem-Based Management:** Implement strategies that prioritize the ecological health of the

In [166]:
# Compile all answers together
together = ""
for index, answer in enumerate(answers):
    together += f"## Response from competitor {index+1}\n\n"
    together += answer
    together += "\n\n"

In [167]:
print(together)

## Response from competitor 1

Balancing the rights of indigenous communities with the goals of economic development in the context of natural resource extraction requires a multifaceted strategy that prioritizes respect, collaboration, and sustainability. Here are several effective strategies:

1. **Free, Prior, and Informed Consent (FPIC):** Ensure that indigenous communities are fully informed about potential resource extraction projects and have the right to give or withhold consent before these projects proceed. This respect for autonomy helps build trust and ensures that indigenous voices are integral to decision-making.

2. **Stakeholder Involvement:** Create inclusive platforms for dialogue among all stakeholders, including government agencies, businesses, and indigenous groups. Collaborative decision-making processes can lead to better outcomes and greater acceptance of development projects.

3. **Ecosystem-Based Management:** Implement strategies that prioritize the ecologica

In [168]:
judge = f"""You are a judge. You are judging competition between {len(competitors)}. You have the following answers to the question: {question}\n\n

Your job is the evaluate each response for clarity and strenght of argument , and rank them in order of best to worst. Respond with Json , and only JSON, with 
the following format:
{{ "results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the response from each competitor:

{together}

Now respond with the JSON  with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks
"""

In [169]:
print(judge)

You are a judge. You are judging competition between 4. You have the following answers to the question: What is the most effective strategy for balancing the rights of indigenous communities with the goals of economic development in the context of natural resource extraction?



Your job is the evaluate each response for clarity and strenght of argument , and rank them in order of best to worst. Respond with Json , and only JSON, with 
the following format:
{ "results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}

Here are the response from each competitor:

## Response from competitor 1

Balancing the rights of indigenous communities with the goals of economic development in the context of natural resource extraction requires a multifaceted strategy that prioritizes respect, collaboration, and sustainability. Here are several effective strategies:

1. **Free, Prior, and Informed Consent (FPIC):** Ensure that indigenous communities 

In [170]:
judge_messages = [{"role": "user", "content": judge}]

In [171]:
openai = OpenAI(api_key=openai_api_key)
response = openai.chat.completions.create(  
    model='gpt-3.5-turbo',
    messages=judge_messages
)

results = response.choices[0].message.content
print(results)

{"results": ["competitor 1", "competitor 2", "competitor 3", "competitor 4"]}


In [173]:
result_dict = json.loads(results)
ranks  = result_dict["results"]
for index, result in enumerate(ranks):
    competitor_index = int(result.split()[-1]) - 1
    competitor_name = competitors[competitor_index]
    display(Markdown(f"### Rank {index+1}: {competitor_name}"))
        

### Rank 1: gpt-4o-mini

### Rank 2: claude-3-7-sonnet-20250219

### Rank 3: gemini-2.0-flash

### Rank 4: llama3.2